## Trace Capture

Like with the last lab, we'll be attacking a jittery AES implementation. This time, however, the jitter will be dispersed throughout the operations, making it much harder to resynchronize.

In [ ]:
#SCOPETYPE = 'OPENADC'
#PLATFORM = 'CWLITEARM'
#CRYPTO_TARGET = 'TINYAES128C'
#SS_VER='SS_VER_1_1'

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 EXTRA_OPTS=JITTER_2 SS_VER=$3

In [ ]:
cw.program_target(scope, prog, "../../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM))

In [ ]:
from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic()
trace_array = []
textin_array = []


proj = cw.create_project("traces/Lab_Resync_WORSE.cwp", overwrite=True)
scope.adc.samples = 5000
N = 1000
for i in trange(N, desc='Capturing traces'):
    key, text = ktp.next()
    trace = cw.capture_trace(scope, target, text, key)
    if not trace:
        continue
    
    proj.traces.append(trace)
proj.save()

In [ ]:
scope.dis()
target.dis()